In [1]:
# load the sequence and probabilities into memory
sequence_filename = "chr22.maf.ancestors.42000000.complete.boreo.fa.txt"
probabilities_filename = "chr22.maf.ancestors.42000000.complete.boreo.conf.txt"

with open(sequence_filename) as f:
    sequence = f.readline()

with open(probabilities_filename) as f:
    probabilities = f.readline().split()

# make all probabilities float
n = len(probabilities)
for i in range(n):
    probabilities[i] = float(probabilities[i])

In [2]:
# set random seed
import random

random.seed(1)

In [3]:
# generate test sequence

In [4]:
def get_test_sequence(length,prob_insertion,prob_deletion):
    nucleotides = ['A', 'G', 'C', 'T']
    
    # get random starting position
    
    len_sequence = len(sequence)
    # len of probabilities should be the same
    
    start_index = random.randrange(len_sequence-length-1)
    
    query = list()
    
    for i in range(start_index, start_index+length):
    
        insertion_deletion_prob = random.random()
        
        if insertion_deletion_prob < prob_insertion:
            # if we are within probability of getting an insertion
            nucleotide = nucleotides[random.randrange(4)]

        elif insertion_deletion_prob < prob_insertion + prob_deletion:
            # if we are within probability of getting a deletion
            nucleotide = ""
        
        else:
            # if there is no indel
            
            if (probabilities[i] > random.random()):
                nucleotide = sequence[i]
            else:
                temp_nucleotides = ['A', 'G', 'C', 'T']
                temp_nucleotides.remove(sequence[i])
                nucleotide = temp_nucleotides[random.randrange(3)]

        # add to our list representing the sequence
        
        query.append(nucleotide)
    
    return ["".join(query),start_index]

In [5]:
# test_name = "test_sequence"
# new_filename = test_name + ".fa"

# length = 300
# prob_insertion = 0.01
# prob_deletion = 0.01

# test_sequence_list = get_test_sequence(length,prob_insertion,prob_deletion)

# with open(new_filename, 'w') as f:
#     f.write(">" + test_name + " " + str(test_sequence_list[1]) + " " + str(length) + "\n")
#     f.write(test_sequence_list[0])

In [6]:
import blast
import os

In [7]:
def test_probabilistic_blast(test_name,iterations):
    
    pblast = blast.PBlast()
    
    exact_index_matches = 0
    contains_match = 0
    
    cur_dir = os.getcwd()
    os.mkdir(test_name)
    
    for i in range(iterations):
        
        new_filename = test_name +  str(i) + ".fa"
        new_path_to_file = os.path.join(cur_dir, test_name, new_filename)

        length = 400
        prob_insertion = 0.005
        prob_deletion = 0.005

        test_sequence_list = get_test_sequence(length,prob_insertion,prob_deletion)

        with open(new_path_to_file, 'w') as f:
            f.write(">" + test_name + " " + str(test_sequence_list[1]) + " " + str(length) + "\n")
            f.write(test_sequence_list[0])
        
        # call to probabilistic blast
        pblast.load_query_file(new_path_to_file)
        
        # compare with start and end index locations
        result = pblast.probabilistic_blast()
        
        if test_sequence_list[1] == result[0] and result[1] - result[0] == length:
            exact_index_matches += 1
        elif test_sequence_list[1] <= result[0] and result[1] - result[0] >= length:
            contains_match += 1
    
    pblast.exit()
    
    return [exact_index_matches,contains_match,iterations]
        

In [8]:
# results = test_probabilistic_blast("third_test",100)

In [9]:
# test 1: 0 10 100
# test 2: 0 4 100
# test 3: 0 45 100

# results

In [16]:
def test_probabilistic_blast(test_name,pblast):
    
    exact_index_matches = 0
    contains_match = 0
    
    cur_dir = os.getcwd()
    test_dir = os.path.join(cur_dir, test_name) 
    
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(test_dir):
        for file in f:
            if '.fa' in file:
                files.append(os.path.join(r, file))
    
    for file in files:
        
        #read first line of fasta file
        with open(file, 'r') as f:
            info = f.readline().split()
            start_index = int(info[1])
            length = int(info[2])
        
        # call to probabilistic blast
        pblast.load_query_file(file)
        
        # compare with start and end index locations
        result = pblast.probabilistic_blast()
        
        if start_index == result[0] and result[1] - result[0] == length:
            exact_index_matches += 1
        elif start_index <= result[0] and result[1] - result[0] >= length:
            contains_match += 1
    
    iterations = len(files)
    return [exact_index_matches,contains_match,iterations]

In [ ]:
pblast = blast.PBlast()

pblast.set_blast_params(gapped_stop = 0)

test_probabilistic_blast("third_test",pblast)

pblast.exit()


total_seeds:  267
stopped before ungapped extension:  4
stopped before gapped extension:  153
HSPs:  110
BEST SCORING ALIGNMENT: 
578334
578732

query: 	CAGTTTATAAAGTACCAAATGAGGCAGGGCTCAAAAGCCCAAGTGCGTGT-
HSP: 	GGCGGGGCTGTGCCAGTTGTGGAATTAATCCTGCTGGCCGGGAGGGAGGCAGTAGCGAGCGGTAGGGACTGCGGGACTGGAGCCAGCTGGAAGGCAGCCCGGCCAGTGGTCATCATGTGGGAACTCTGTCTAGTGTCGCCAGAATTTCCAGTTGCTCAAGTGCGGCAGGAGGTCTGCTGTGGTGTATTCTTTTAGCTGGTCCCAGCGATGGTCAAAGGCACTGTACCGGACTGTGGGATGTTTTGGCCACGGGCTGCCAGTTTGTGACCTGTTGGTGAGGTATGTCTTGGTAAACCTCTCTGATCCTTTGTCTGCCCCTCCCTGCAAACCCATCATGGAGAGGGGAGGT

db: 	-AGTTTATAAAGTACCAAACGAGGCAGGGGTCAAAAGCCCAAGTGCCTGTG
HSP: 	GGAGGGGCTGGGCAAGTTGTGGAGTTAATCCTGCTGGCTGGGAGGGAGGCAGTAGGGAGTGGTAGGGACTGTGGGACTGGAGCCAGCTGGCGGGCAGCCGGGCCAGTGGTTATCATGTGGGAACTCGGTCTAGTGTGGCCAGAATTTCCAGTTGCTCAAGAGAAGCAGGAGGTCTGCTGTGTTGTATTTTTTTAGGTGGTCCCAGCAAGGGTCAAAGGGACTGTACAGGACTGGGGGCTGTTTTGGCCTCGGGCTGCCAGTTTGTGACCTGTTGGGGAGGATTGTCTTTGTTATCATCTCTGAACCTTTGTCTGCCCCTCCCTGCAAACCCATCATGGAGAGGGGAGGT
326.23999999999967
total_

total_seeds:  327
stopped before ungapped extension:  0
stopped before gapped extension:  134
HSPs:  193
BEST SCORING ALIGNMENT: 
245340
245739

query: 	
HSP: 	AGTTGCCTTAGCTATTCTTCCTCTGGCCTGTCTTTTTGTCTTCTCTCCTATCTGGCCTTGCCTTTGCTTCTCTAAGGTTTAGTCCCCGTCTTCC
gap: 	-TGAGAGGCGGGTAGACGTACTGTCAAGCCCTACAGCACTGATCTAATGGATGTACCAGAGAACTCTTTCAAACTTTGACGACAAATAATCCTTTTGTGAGTCTCTGCCATCTTTAACACATGGCTCCCACGGCCTCTTTGGTCAGCCAGCAGAGAGGGGAAGAGAATAAAGATCTCACCTGGGAATTTTTCCGGGGCCAGGCCTGGAAGCCGTGTACATTACTTCTTGTCACATGCCATAGGCCAGAATTGAGTGTGGTAGGCTGATAATTGCTCCCAAAGATATATCCATGTCATAATCCTCAG
db: 	
HSP: 	AGTTGCCTTAGCTCTTCTTCCTCTGGCATGGCTTTTTGTCTTCTCTCCTCTCTGGCCTTGCCTTTGCTTCTCTTAGGTTTAGTCCCCGTCTTCC
gap: 	ATGAGAGGGGGGTAGAGGTACTGTCAAGCCCTACAGCACTGATCTAATGGATGTACCAGAGAACTCTTTCAAACTTTGAATACAAATAATCCTTTTGTGTGTCTCTGCCATCTTTAACACATGGCTTCCACGGCCTCTTTGGATAGCCAGCAGAGAGGGGAAGAGAATAAAGATCTCACCTGGGAAGTTTTCATGGGCCAGGCCTGGAAGCTGTGTACATTACTTCTGGTCACATGCCATAGGCCAGAACTCAGTGTGGTAGGCTGATAATGGATCCCAAAGATATATCCATGTCCTAATCCTCA-
367.89999999

total_seeds:  443
stopped before ungapped extension:  3
stopped before gapped extension:  316
HSPs:  124
BEST SCORING ALIGNMENT: 
448274
448670

query: 	ACACATGCTGGAAAGCCACCCAGGAGGCCGCCGCTGGGATGCAGGAGGGTAGAGACGGTGGCCCAGCCTCAGAAGGTGGCAGAGAAAACCGGGAAGCAGCCCAGGGCTGGCCATGGGGGAGGCCAGAGGCCCAGGGGGCTGCAGGCCAGTGCCCACCCCATTGTGGC-
HSP: 	CTCCCCCTCTGAGTCCCCTTCCTCCACCTGCCAGCCCCATCCCCTACCCA
gap: 	-GCCAAGACCTCGGAAAAGAGTTGCCTCAAACTGGCTGTTAAGGGCTCAGCGGGCGGTGGAGGGAGGGTGGCCTCTCCCGGGGCCACCATGTTCAAATACAGGTCCAGGCCCAGAGTAACAGGA-ACCCACTCAGCCATCCAGTAGAGGCCCC-GGCTGCCTGCTGGTCCCTTACCTTGGCC

db: 	-CACATGCTGGAAAGCCACCCAGGAGGCCACCGCGGGGATGCAGGAGGGTGGAGATGGTGGCCTAGCCTCAGACGGTGGCAGAGAAAACCGGGAAGCAGCCCAGGGCTGGCCATGGGGGAGGCCAGAGGCTCTGGGGGCTGCAGGCCAGTGCCCACCCCAATGTGCCA
HSP: 	CTCCCCCTCTGAGTCCCCTTCCTCCCCCTGCCTGCCCCATCCCCGACCCA
gap: 	AGCCAAGCCCTCGGAAAAGGGTGGCCTCAAACTGGTTGGAAATGGCTCAGTGGGCGGTGGAGGGAGTGTGGACTCACCAGGGGCCACCAGGTTCAAATACAGGTCCTGGCCCAGAGTAACAGGAGACCCACTCAGCCATCCGGCGGAGGCCCCAGGCTGCCTGCTGGCCCCTTCCCTT-G--
334.3399999

total_seeds:  316
stopped before ungapped extension:  0
stopped before gapped extension:  66
HSPs:  250
BEST SCORING ALIGNMENT: 
387845
388244

query: 	
HSP: 	AACCATCTTCCAATCAGGCTGAAATTCAAGTGTTCCAATGAACTTCGACCCTTTCCTACAGAACAGCTTACATGAACATACTGCAATAAGGATATTTTCCCCAGTAGCTTAACTACCCCACAATGAAGCATCAGAGAACAAGCCACAGATACACCAATAACTCACATTATAGTTCATCTTCCTGCTGGGTCGTGCGCATTACCTACAAAGCAAACACAGCTCTAAAGCAGAAAGTACTGATTGTGTTTGTCGC
gap: 	-ATGAGAACCTTTTCACCCATACTGCACGTACCATGGGTATTAAAGGACTTCCAACAGCATAACTAATAATCAAAAAACTATCCAGTCTAAGTGCAATGCGACCAAATTATGCTCTACGAACTATGAAAGCCTTGATTTCAGTATAT
db: 	
HSP: 	AACCATCTTCCAATCAGGCTGATATTCAAGTGTTCCAATGAACTTCTACCCTTTCCTATAGAACAGCTTACATGTGCATACTGCAATAAGGATATTTTCCCCAGTAGCTTAACTTCCCCACAATGAAGCATCAGATAACAAGCCACAGATACACCAATAACTCACGTTATAGTTCATCTTCCTGCTGGGTCGTGCGCATTACCTACAAAGCAAACACAGCTCTAATGCAGAAAGTACTGATTGTGTTTGTCGC
gap: 	CATGAGAACCTTTTCACCCATACTGCACGTACCATGGGTATTAAAGGACTTTCAACAGCCTAACTAATAATCAAAAAACTATCCAGTCTAAGTGCAATGTGACCAAATTATGCTCTACTAACTATGAAAGCCTTGATTTCAGTATA-
391.339999999

total_seeds:  612
stopped before ungapped extension:  1
stopped before gapped extension:  291
HSPs:  320
BEST SCORING ALIGNMENT: 
299476
299874

query: 	CACGGGACCCTCAACCCTCCTCCCCTGCCCCCGGCCCCG-CAGCCGCTCTCACCTTCTCCACCTCACCCAGGAAGGCGTCAGTCAGGTCTCGGGGTGGCTGCGCCGGGTCCCGGGTCCTCCTGTGCTCAGTGACCAGCTCATCCAGCAGGGCCATGAAGGCCTTCTGCCCAGGGAAGACCTTGTCGGCCAGCCCCGGGATGCGCAGGAGCACGGGGATCGCTTTCAGCACCTGTGAAGGCACAGAAAGGGGCCTCGGTCCTTCCTGTGCTCACGAGGTCCCCTGGACCA-TG
HSP: 	TCAGGGCCCAGCCTTCACCAGGGCCTGCCGGTCCCCACCAGTGACCTGTTTCCTCCCCAAGTTCAGGCCTCCACCCTCTCTCCTGGCAAGGACAAACCTAAAATCTA
gap: 	A

db: 	--CGGGGCCCTCAACCCACCTCCCCTGGCCCCGTCCCTGGCAGCCGCTCTCACCTTCTCCACCTCATCCAGGAAGGCGTCAGTCAGGTCTCGGGGTGGCTGGGCCGGGTCCCGGGTCATCCTGTGCTCGGTGACCAGCTCATCCAGCAGGGCCATGAAGGCCTTCTGCCCGGGGAAGACCTTGCCGGCCAGCCCCGGGATGCGCAGGAGCACGGGGACCGCGTTCAGCACCTGTGACGGCACAGAAGGGGGTCTCGGTCCTTCCTGTGCTCAGGTGTTCACCTGGACCAGTC
HSP: 	TCAGGTCCCAGCCTCCTCCAGGGCCTGCCCGTCCCCACCATTGACCTGGTTCCTCCCCAAGTGCAGGCCTCCACCCTCTCTCCTGGCAAGGATAAACCTAAAATCTA
gap: 	C
347.9799999

...

total_seeds:  611
stopped before ungapped extension:  5
stopped before gapped extension:  323
HSPs:  283
BEST SCORING ALIGNMENT: 
86849
87248

query: 	
HSP: 	AGGACTGGTGAGGCTCATAAGACTCTGTGTGGCCTCCTCTGTCCCCTCTAGATGCAGCGGGTATGTGGGGGGAGGGGGTAGAAGAGATAATGGGGCAGGTTATTTTCTGCCCAAGCTCCCAGAATGTGGGGGTTAGCCCTTTACCCCCTGCCCTGCCAGGTTCACCAGACACTGTCTGTGGAACTGGACCAAGTATTGAAGGCCCTCAGCTTTCCAAAGAAAAAG
gap: 	-CTGCGCTGCTCTCAGGTGTGGGCCCAGGGGCCTCCAGCAGGAACAGAAAAGCTTTTCCCCTAGATTCTGAGATGGGACCACCCCACCCAGACCAGCCTGTGGTCACCCCATAATTATCTCCATTGTTTACCATCCACTGACAGAGCTGGGTGGGAGGTGGGGAATGAGGCAGGA
db: 	
HSP: 	AGGACTGGTGAGGCTCAGAAGACTCTGTGTGGCCTCCTCTGTCCCCTCTGGATGCAGCGGGTATGTGGGGGGAGGGGGTAGAAGAGATATTGGGGCAGGTTATTTTCTGCCCAAGCTCCCAGAATGTGGGGGTTAGCCCTTTACCCCCTGCCCTGCCAGGTTCACCAGACACTGTCCGTGGAACTGGACCAAGTATTGAAGGCCCTCAGCTTTCCAAAGAAAAAG
gap: 	GCTGCGCTGCTCTCAGGTGTGGGCCCAGGGGCCTCCAGCAGGAAGAGAAAAGCTTTTCCCCTAGATTCTGAGATGGGACCACCCCACCCAGACCAGCCTATGGTCACCCCATAATTATCTCCATTGTCTCCCTTCCTGTGACAGAGCTGGGTGGGAGGTGGGGAAGGAGGCAGG-
388.0399999999

total_seeds:  229
stopped before ungapped extension:  0
stopped before gapped extension:  127
HSPs:  102
BEST SCORING ALIGNMENT: 
256647
257044

query: 	GCCAGTCCCTTCTTCTTCATCTTCACTGCTATCACCCCCAGTCCCAGCCTCCCTGTCTCTCTCCTGGACCCTTGCACAGC-GATTTCAGGCTCACCGTATACCTGCTAGGGTCCAGCCAGAAGACTGAACCCACATCAGTTATTTGAACTGAGAGTTGTGCAACCGTAAGCACTATCCAATTCCAGAACATTTTCTATGTTTATTCTAGACCTATCTTC-
HSP: 	TGTAGAATTTTTTCTAGAAATGCATATATATATTTTTAAAGACCCGATCTGTGAGAGACATAGTGTGCTTTAGGGTTTTCTCAGATTTGCTTTCAAATATTCCGGCTCACACACCAAAAAAGATTGTGCGGGCAGAGACATGAAATAATGGCACAAGTCGATGGTTGTTAGAGCTGGATG

db: 	--CAGTCCCTTCTTCTTCATCTTCACTGCAACCACCCCCTGTCCCTGCCTCCCTGTCTCTCACCTGGACCCTTGCACAGCTGATTCCAGGCTCCCTGTGTACCTGTCAGGGTCCAGCCAGGAGACAGAAACCACATCAGTTATTTGAACAGAGAGTTATGCAACCATAAGCACTATCTAATTCCAGAACATTTTCTTTGTTCATTGTAGACCTATCTTCT
HSP: 	TTTAGAATTTCTTCTTGAAACTCATATATATATTTTTAAAGACCCTATCTGTTAGAGACATAGTGGTGTTTAGGGTTTTCTGAGATTTGCTTTCAAATACTCCAGCTCACACGCCAGAAAAGTGTGTGCGGGGAGAGAGATGAAATAATGGCACAAGTTGATGGTTGTTGGAGCTGGGTG
323.1899999999999
total_s

total_seeds:  349
stopped before ungapped extension:  1
stopped before gapped extension:  256
HSPs:  92
BEST SCORING ALIGNMENT: 
292755
293153

query: 	
HSP: 	GGCGGGGTGGCCACGTGGGCGTGGCCACATGTAGGCAATGGCCTCGCCCCGGTTCGCCAACACTGTGCTCCCCCCCTCCCCCCGTGCACCGGAACAGGGCCCACTCTGCGAGCGCCCGCCTGGCCGGG
gap: 	-CAGCCTTGAGGCCC-ACCTCGCCTCTGCCCACCCTGACCGCCCGTCCGCTCAGGGCAGACCCCGCCTACCCCGCCCCGCCCACGTTGAGCCCCCCCCTTCCACTACCTCCAACCGAGGGTCACCACCCAGTCCCGCCCCCCCGGCGCCTGCCTG-TCCTCGGC-GTTGCGCCGCCCTCCTCCCTCT-CACCCTCGGCGCAGGCTGCTTCTTCGACCTCCCGCCCGC-TTTGCTGGCTTCGCTAGTGCCGCGCACTTACCGCGAACCTGACCCCGC
db: 	
HSP: 	GGCGGGGCGGGCACGTGGGCGTGGCCACATGTAGGCCTTGGCCCCGCCCTCTCTCGCCGACCCTGAGCTCCCCCCCTCCCCCCGTCCGGCGGCACAGGGCCCACTCTGCGCCCGCCCGCCTGGCTGGG
gap: 	GCAGTCTTG-GGCCCCACCTCGCCTCTGCCCACCCTGACCGCCCCTCCACTCAGGGAAGATCCCGCCTGCCCCGCCCCGCCCACACTGAGCCCCCCGCTTCCACCGACTCCAGCAGAGGGACGCGTCCCAGGCCCGCCCCGCCGGCCCCTGCCGGGT-CTGGGCAG-TGCCCCGCCCTCGTCCCCCTCCACCCTCGGCGCAGGCTCCTTCCTCGCCCTGCCGCCC-CACTTGCTGGCTTCTCTAGTGCTCCGCACATCCCGCGACCCTG-CCCC-C
268.0

total_seeds:  250
stopped before ungapped extension:  0
stopped before gapped extension:  134
HSPs:  116
BEST SCORING ALIGNMENT: 
107235
107631

query: 	TGGGGTTGAGAGAGAGCTAGGGGTT-AAATGAGCCAGGCCTTGGCAATGGACTCAATGTGGAGAGGGCGCGGTGGGCGCAGGTGGTGCTACTTGGTTTCTGCCCCGCTGTTCATCAGCACAGGGACTCTGGC-GTGGACTGGGTTTTCGGGAGATGGAGCCATGCCGAG-
HSP: 	TTTGGGCGCATTTGGGACATCCTAGTGGAGGTGTCCGACAGGCGTTTAGATATTGAAATGCAGGTTTGAGGGGTAACTGACGATATAGGCGTAACAGAAGCCATGGGTTAGAAGAGCCAGTCTGGAGAGAGCGGAGGGATAAGAGCTAAGGGCCTGGGGATAGCAGTCGAAGAAGGCCCCCACAGAAGTCCTCTTCCTCCCTCCCTTCTTCAGTTCGCCCTCCGTCCCAC

db: 	---GTTTGAGAGAGACTTAGGGGTTAAAATGAGCCAGGCCTTGGCGATGGACTCAATGTGGAGAGGGAGTGGCGGGTGCAGGTGGGGCTCCTTGGTTTCTGCCCTGCTGTTCATCAGGACAGGGACTCTGGAAGAGGACTGGGTTTTGGGGAGATGGAGCCATGCCGAGT
HSP: 	TTGGGGCGCATTTGGGACATCCTAGTGGAGGTGTCAGACAGGCATTTAGATATGGAGATGCAGGTTTGAGGGGTAACTGACCATATAGGCGTAACAGAAGCCGTGGGTTAGATGAGCCAGTCTGGAGAGAGCGGAGGGAGAAGAGCCAAGGGCCTGGGGATTGCAGTCGAAGAATGCCCCCACCGAAGTCCTCGTCCCCACCGCGGTCTTCTGTTTGACCTCTGTCCCAC
324.68999999999994
total_

total_seeds:  351
stopped before ungapped extension:  4
stopped before gapped extension:  238
HSPs:  109
BEST SCORING ALIGNMENT: 
497636
498035

query: 	
HSP: 	GGCCACCTGGGCTCCCGTCATTCCCGAAAAGCTCCTCCTTGGATGTCTCTAGGTCCCACCCCCCACTCTCTGACCACAGCCCCGCCCTGCCCAAGCCTCCTCCCAGGGAAGGATAGTGAGCGCTGGACTCGGCAAGGGGGGCACCGAGCCCCTGGCCTCCACGGCTCCCTGCACGGGATCATGGTCAGGGCTCCGCTCCCACCAGCCTCCGCCACCAGACGGGAGCACCTAGAGGGCAGGGTCCGTGCCTAGTTTGTCTTTGGGTTCCTGAACCTACTGCTGTGCTTGGCACAACGGGATCGGCACCAAATGAGCTATGCCGAGGGGGCTCCTCCCTCACTAGGGTGTGTCC
gap: 	-CGTTCCTCACTCACTCTTGGCCCAGCTACGAGTGGCCTGACCTGATG
db: 	
HSP: 	GGCCACCTGGGCTCCCGTCATTCCCCAAATGCTCCTCCTTGGAGGTCTCAAGTTCCCACCACCCACTCTCTGACCACAGCCCCGCACAGCCCAAGCCTCCTCCCAGGGAAGGCGAGTGAGCCCCGGGCTCGGCTGGGAGGGATCCGAGCCCCTGGCCTCCACGGCTCCCTGCACCGGATCATGGTCGGAGCCCCGCTCCCACCAGCCTCCCCCACCGGATGGGAGCCCCTTGAGGGCAGGGACCGTGCCTGGTTTGTCTCTGGGTCCCTGAACCTACAGCTGTGCTTGGCACAAAGAGAGCGGCACCAAATGAGCTATGGTGAGGGGGCTCCTCGCTCACTTGGGTGTGTCC
gap: 	TGGGCCCACACTCACTCTTGGCCCAGCTACGAGCGGCCTGACCAGA-G
320.69000000